<a href="https://colab.research.google.com/github/mr1er0y/Topic-Modelling-Neuro/blob/main/preprocessing_arxiv_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Классификация тенденций в науке
### Набор данных Arxiv

Набор данных Arxiv datasets на платформе Kaggle - это сборник документов, собранных с открытого репозитория научных статей ArXiv.org. Этот набор данных содержит более 1,7 миллиона научных статей, опубликованных на протяжении более чем 20 лет, начиная с 1992 года. В наборе данных представлены работы по таким областям, как физика, математика, компьютерные науки, биология, экономика и многие другие.

Набор данных Arxiv содержит множество метаданных для каждой научной статьи, таких как авторы, заголовок, аннотация, дата публикации, категории и теги. Кроме того, каждый документ представлен в формате текста, что позволяет проводить анализ текстовых данных, такой как тематическое моделирование, классификация и кластеризация.

In [1]:
import dask.bag as db #dask 
import re
import json
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import string
import logging

logging.captureWarnings(True)

input_file = "drive/MyDrive/Data_arXiv/arxiv-metadata-oai-snapshot.json"
output_file = "drive/MyDrive/Data_arXiv/filtered_arxiv_db.csv"

### Загрузка данных

Набор данных arXiv доступен в формате json, и хотя содержащиеся в нем фактические статьи были исключены, его размер составляет более 3 гигабайт, со статьями размер данных содержит более 1 ТБ.  Из-за огромного объема данных необходимо тщательно продумать процессы загрузки и манипулирования ими, особенно при использовании локальных вычислительных ресурсов.

Dask Bag обрабатывает данные поэлементно, что позволяет распараллеливать обработку данных на несколько вычислительных узлов. В отличие от других инструментов для работы с данными, таких как Pandas или NumPy, Dask Bag не представляет данные в виде таблицы или массива, а обрабатывает данные как набор отдельных элементов, каждый из которых может иметь произвольную структуру.

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:

loaded_data = db.read_text(input_file).map(json.loads)

In [4]:
#Check Loaded Data
print("The dataset has {} rows.".format(loaded_data.count().compute()))

The dataset has 2203919 rows.


### Фильтрация и очистка данных
Набор данных содержал более 2 миллионов мета данных, но был отфильтрован до более релевантных данных, которые были определены как темы, исследованные в течение последних 4 лет. Фильтрация была выполнена с использованием Dask Bag и преобразована во фрейм данных Pandas. Данные, созданные или обновленные в течение последних 5 лет, были учтены в процессе фильтрации. Набор данных был сокращен до 600 тыс. (30%) мета данных и некоторые типы данных были изменены для облегчения процессов манипулирования данными.

In [5]:
# Define a function that returns the lastest year the paper was either published or updated. This function will be bases to the bag.filter() method.
def get_last(x):
    get_latest_version = int((x['versions'][-1]['created']).split(' ')[3])
    get_latest_update = int((x['update_date']).split('-')[0])
    last_date = max([get_latest_version, get_latest_update])
    return last_date
                            
# Define a function that retunrs only the relevant attributes
trim_attributes = lambda x: {'id': x['id'],
                  'authors': x['authors'],
                  'title': x['title'],
                  'category':x['categories'].split(' '),
                  'abstract':x['abstract'],
                  'versions':x['versions'],
                  'update_date':x['update_date'],}

# Filter all articles that have ramained relevant or mostly investigated for at least the past 5 years
columns = ['id','category','abstract']
loaded_data = (loaded_data.filter(lambda x: get_last(x=x) > 2019).map(trim_attributes).compute())

# convert to pandas
df = pd.DataFrame(loaded_data)

#check data frame
df.head()

,id,authors,title,category,abstract,versions,update_date
0,0704.0033,"Maxim A. Yurkin, Valeri P. Maltsev, Alfons G. ...",Convergence of the discrete dipole approximati...,"[physics.optics, physics.comp-ph]",We performed a rigorous theoretical converge...,"[{'version': 'v1', 'created': 'Sat, 31 Mar 200...",2022-03-31
1,0704.0038,"Maxim A. Yurkin, Alfons G. Hoekstra",The discrete dipole approximation: an overview...,"[physics.optics, physics.comp-ph]",We present a review of the discrete dipole a...,"[{'version': 'v1', 'created': 'Sat, 31 Mar 200...",2022-03-30
2,0704.0275,George M. Bergman (U.C.Berkeley),Mapping radii of metric spaces,[math.MG],It is known that every closed curve of lengt...,"[{'version': 'v1', 'created': 'Mon, 2 Apr 2007...",2021-10-15
3,0704.0479,T.Geisser,The affine part of the Picard scheme,"[math.AG, math.KT]",We describe the maximal torus and maximal un...,"[{'version': 'v1', 'created': 'Wed, 4 Apr 2007...",2021-01-29
4,0704.0752,Davoud Kamani,Actions for the Bosonic String with the Curved...,[hep-th],At first we introduce an action for the stri...,"[{'version': 'v1', 'created': 'Thu, 5 Apr 2007...",2020-08-21


In [6]:
#Check data frame size
df.shape

(658572, 7)

In [7]:
#Check info and data types
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 658572 entries, 0 to 658571
Data columns (total 7 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   id           658572 non-null  object
 1   authors      658572 non-null  object
 2   title        658572 non-null  object
 3   category     658572 non-null  object
 4   abstract     658572 non-null  object
 5   versions     658572 non-null  object
 6   update_date  658572 non-null  object
dtypes: object(7)
memory usage: 35.2+ MB


In [8]:
#Check null values
df.category.isnull().sum()

0

In [9]:
#Define function to generate new features

def last_creation_date(x):
    last_date = (x[-1]['created']).split(' ')
    last_date = last_date[1] + ' ' + last_date[2] + ' ' + last_date[3] 
    return last_date

def split_categories(example):
    # sequences = example.split(' ')
    return np.unique([el.split('.')[0] for el in example])

def abstract_prep(data):
    data = data[df['abstract'].str.contains('paper has been withdrawn') == False]
    data['abstract'] = [a.strip() for a in data['abstract']]
    data['abstract'] = data['abstract'].str.replace('\n', ' ', regex = False).str.replace('\t', ' ', regex = False).str.replace(r'\s\s+', ' ', regex = True)
    data['abstract'] = data['abstract'].str.replace('([.,!?()])', r' \1 ')
    return data

In [10]:
# Generate new features by traforming the old category attribute and filter data
# df = abstract_prep(df)
# df['update_date'] = pd.to_datetime(df['update_date'])
# df['created_date'] = pd.to_datetime(df['versions'].apply(lambda x: last_creation_date(x=x)))
df['main_category'] = df['category'].apply(lambda x: split_categories(x))
df = df.reindex(columns=['id', 'title', 'authors', 'category', 'main_category', 'created_date', 'update_date', 'abstract', 'versions'])

In [11]:
df.head()

,id,title,authors,category,main_category,created_date,update_date,abstract,versions
0,0704.0033,Convergence of the discrete dipole approximati...,"Maxim A. Yurkin, Valeri P. Maltsev, Alfons G. ...","[physics.optics, physics.comp-ph]",[physics],NaN,2022-03-31,We performed a rigorous theoretical converge...,"[{'version': 'v1', 'created': 'Sat, 31 Mar 200..."
1,0704.0038,The discrete dipole approximation: an overview...,"Maxim A. Yurkin, Alfons G. Hoekstra","[physics.optics, physics.comp-ph]",[physics],NaN,2022-03-30,We present a review of the discrete dipole a...,"[{'version': 'v1', 'created': 'Sat, 31 Mar 200..."
2,0704.0275,Mapping radii of metric spaces,George M. Bergman (U.C.Berkeley),[math.MG],[math],NaN,2021-10-15,It is known that every closed curve of lengt...,"[{'version': 'v1', 'created': 'Mon, 2 Apr 2007..."
3,0704.0479,The affine part of the Picard scheme,T.Geisser,"[math.AG, math.KT]",[math],NaN,2021-01-29,We describe the maximal torus and maximal un...,"[{'version': 'v1', 'created': 'Wed, 4 Apr 2007..."
4,0704.0752,Actions for the Bosonic String with the Curved...,Davoud Kamani,[hep-th],[hep-th],NaN,2020-08-21,At first we introduce an action for the stri...,"[{'version': 'v1', 'created': 'Thu, 5 Apr 2007..."


In [12]:
df = df[df["abstract"].apply(len) > 150]

In [17]:
df[df["main_category"].apply(len) == 1]["category"].value_counts()

[cs.CV]                                 23812
[quant-ph]                              14422
[cs.CL]                                 10418
[math.AP]                               10389
[cond-mat.mtrl-sci]                      8885
                                        ...  
[cs.DC, cs.AR, cs.DS, cs.PF]                1
[cs.CL, cs.AI, cs.IR, cs.PL]                1
[cs.AI, cs.DL, cs.LG, cs.LO]                1
[math.AG, math.DS, math.GR]                 1
[math.GT, math.CO, math.MG, math.SG]        1
Name: category, Length: 9265, dtype: int64

In [14]:
#Save filtered and cleaned data into a csv filed
# df.to_csv(output_file, index = False)